## 超参数的概念
1. 超参数就是模型训练中需要人为设置的参数（英文为Hyberparameter）
2. 我们之前实现KNN分类任务的时候，默认都是选择K=5，p=2的明氏距离(欧式距离)，K中各个点的投票权重都是一样的
3. 所以在KNN中就有三个超参数，那超参数要如何选择呢，看下面。

## 超参数的选择 
1. 经验值：一种是来自于经验的选择
2. 参数搜索：一种是遍历所有参数情况，找出最好的参数组

## 超参数选择的代码实现(参数搜索)

### 1.生成数据

In [1]:
import numpy as np
from sklearn import datasets

In [5]:
# 我这里采用sklearn的iris数据集
iris = datasets.load_iris()

In [8]:
X = iris.data
y = iris.target

### 2.数据集划分

In [9]:
from sklearn.model_selection import train_test_split 

In [10]:
# stratify=y,是类别平衡的意思，可以保证我们训练集的标记比是1：1：1的，你可以尝试去掉，用Counter看看比例
x_train,x_test, y_train, y_test = train_test_split(X,y,random_state=233,train_size=0.7, stratify=y)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((105, 4), (45, 4), (105,), (45,))

### 3. 参数搜索, 生成最优超参数

In [30]:
from sklearn.neighbors import KNeighborsClassifier

In [31]:
## 我们可以遍历各个参数，然后看最优模型的超参数是什么
## 定义一个函数
# uniform代表权重相同，distance代表距离越远，权重越低
def param_search(max_k, weights: ['uniform','distance'], max_p, x_train, y_train, x_test, y_test):
    best_score = -1
    best_k = -1
    best_weight = ''
    best_p = -1
    for k in range(1, max_k):
        for weight in weights:
            for p in range(1, max_p):
                knn_neighbors = KNeighborsClassifier(n_neighbors=k, weights=weight, p=p)
                knn_neighbors.fit(x_train,y_train)
                score = knn_neighbors.score(x_test, y_test)
                
                if score > best_score:
                    best_k = k
                    best_p = p
                    best_weight = weight
                    best_score = score
    return best_score, best_k, best_weight, best_p

In [35]:
## 调用函数
best_score, best_k, best_weight, best_p = param_search(
    max_k=10,
    weights=['uniform','distance'],
    max_p=7,
    x_train=x_train, 
    x_test=x_test, 
    y_train = y_train,
    y_test=y_test
)
best_score, best_k, best_weight, best_p

(1.0, 5, 'uniform', 2)

## sklearn的超参数搜索

In [36]:
## 上面我自己写的超参数搜索，sklearn已经实现过了，我们可以借助sklearn实现
from sklearn.model_selection import GridSearchCV

In [34]:
param = {
    'n_neighbors': [k for k in range(1,10)],
    'weights': ['uniform','distance'],
    'p': [p for p in range(1,7)]
}

In [37]:
## n_jobs是并行计算数量，-1代表由GridSearchCV自己决定，因为我们参数之间没有什么关系，所以可以交给GridSearchCV自己决定
grid = GridSearchCV(
    estimator= KNeighborsClassifier(),
    param_grid=param,
    n_jobs=-1
)

In [38]:
# 传入训练集进行训练
grid.fit(x_train,y_train)

GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'p': [1, 2, 3, 4, 5, 6],
                         'weights': ['uniform', 'distance']})

In [41]:
# 最优超参数，我们可以看到这里的最优超参数和我们的自己得到的最优超参数不同，
# 这是因为我们并没有给grid传入测试集，所以这个是他自己根据我们的样本计算的
grid.best_params_

{'n_neighbors': 9, 'p': 2, 'weights': 'uniform'}

In [44]:
# 最优的准确率, 准确率不一样也和评价算法有关
grid.best_score_

0.961904761904762

In [45]:
# 最优模型
grid.best_estimator_

KNeighborsClassifier(n_neighbors=9)

In [47]:
# 用最优模型预测测试集结果
grid.best_estimator_.predict(x_test)

array([2, 2, 0, 1, 1, 1, 2, 0, 2, 0, 0, 1, 0, 2, 1, 1, 0, 2, 2, 1, 0, 1,
       1, 2, 2, 0, 0, 1, 1, 0, 2, 2, 0, 1, 1, 2, 1, 1, 0, 0, 0, 2, 0, 1,
       1])

In [49]:
# 计算准确率
grid.best_estimator_.score(x_test,y_test)

0.9555555555555556